In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


2024-09-06 17:53:07.265800: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-06 17:53:07.283419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 17:53:07.304279: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-06 17:53:07.310871: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-06 17:53:07.325124: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Define image size and batch size
IMG_SIZE = (224, 224)  # Image size for VGG16
BATCH_SIZE = 32

# Paths to your dataset
train_dir = '/home/esidserver/datasets/testing_cls/train/'
val_dir = '/home/esidserver/datasets/testing_cls/val/'
test_dir = '/home/esidserver/datasets/testing_cls/test/'

In [3]:
# Data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)


Found 5193 images belonging to 4 classes.
Found 1484 images belonging to 4 classes.
Found 740 images belonging to 4 classes.


In [4]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [5]:
# Load the VGG16 model pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (don't train them initially)
base_model.trainable = False

# Create a new model on top of it
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 output classes
])

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Set up a checkpoint to save the best model
checkpoint = ModelCheckpoint('best_vgg16_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')

#Set up a earlystop to save the best model
early_stopping=EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True)

# Set up a checkpoint to save the best model
#checkpoint = ModelCheckpoint('best_vgg16_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

2024-09-06 17:53:09.601712: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21783 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


In [6]:
#pip install --upgrade tensorflow tensorflow-gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



Num GPUs Available:  1


In [7]:
#auto early stop if the efficiecy is not improving for 50 epochs
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=250,  # Adjust the number of epochs based on your needs
    callbacks=[checkpoint]
)

# Unfreeze the base model for fine-tuning
base_model.trainable = True




Epoch 1/250


/home/esidserver/miniforge3/envs/tf/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1725619996.559659 2647973 service.cc:146] XLA service 0x77d8a0016a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725619996.559731 2647973 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-09-06 17:53:16.647593: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-06 17:53:17.006092: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


  1/162 ━━━━━━━━━━━━━━━━━━━━ 24:44 9s/step - accuracy: 0.1875 - loss: 1.8969

I0000 00:00:1725620001.354979 2647973 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


162/162 ━━━━━━━━━━━━━━━━━━━━ 80s 439ms/step - accuracy: 0.4586 - loss: 1.2455 - val_accuracy: 0.7425 - val_loss: 0.6442
Epoch 2/250
  1/162 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5000 - loss: 0.9470

...

2024-09-06 19:58:35.941745: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 19:58:35.941853: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554
2024-09-06 19:58:35.972064: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 19:58:35.972199: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554


162/162 ━━━━━━━━━━━━━━━━━━━━ 61s 357ms/step - accuracy: 0.9359 - loss: 0.1715 - val_accuracy: 0.9674 - val_loss: 0.0859
Epoch 248/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step - accuracy: 0.9688 - loss: 0.1253 - val_accuracy: 0.8333 - val_loss: 0.2185
Epoch 249/250


2024-09-06 19:59:36.596180: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 19:59:36.596287: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554


162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 349ms/step - accuracy: 0.9372 - loss: 0.1610 - val_accuracy: 0.9647 - val_loss: 0.0991
Epoch 250/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step - accuracy: 0.8750 - loss: 0.2594 - val_accuracy: 1.0000 - val_loss: 0.0062


2024-09-06 20:00:35.692397: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 20:00:35.715786: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 20:00:35.715882: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554


In [8]:
# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=100,  # Further epochs for fine-tuning
    callbacks=[checkpoint]
)


Epoch 1/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 90s 410ms/step - accuracy: 0.9015 - loss: 0.2601 - val_accuracy: 0.9450 - val_loss: 0.1243
Epoch 2/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9688 - loss: 0.1620 - val_accuracy: 1.0000 - val_loss: 0.0027
Epoch 3/100


2024-09-06 20:02:07.929623: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 20:02:07.929786: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554


162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 346ms/step - accuracy: 0.9436 - loss: 0.1595 - val_accuracy: 0.9688 - val_loss: 0.0866
Epoch 4/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 254us/step - accuracy: 0.9688 - loss: 0.0924 - val_accuracy: 1.0000 - val_loss: 0.0587
Epoch 5/100


2024-09-06 20:03:06.964466: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 20:03:06.964592: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554


162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 349ms/step - accuracy: 0.9534 - loss: 0.1319 - val_accuracy: 0.9660 - val_loss: 0.0774
Epoch 6/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 248us/step - accuracy: 1.0000 - loss: 0.0574 - val_accuracy: 1.0000 - val_loss: 0.0391
Epoch 7/100


...

162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 347ms/step - accuracy: 0.9930 - loss: 0.0195 - val_accuracy: 0.9952 - val_loss: 0.0168
Epoch 92/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 2.5227e-04
Epoch 93/100


2024-09-06 20:46:46.382311: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 20:46:46.382421: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554


162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 352ms/step - accuracy: 0.9943 - loss: 0.0175 - val_accuracy: 0.9925 - val_loss: 0.0242
Epoch 94/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 277us/step - accuracy: 1.0000 - loss: 4.6118e-04 - val_accuracy: 1.0000 - val_loss: 0.0265
Epoch 95/100


2024-09-06 20:47:45.805895: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 20:47:45.806004: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554
2024-09-06 20:47:45.832232: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 1679156592267202918
2024-09-06 20:47:45.832333: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 12293121517622605554


162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 347ms/step - accuracy: 0.9942 - loss: 0.0157 - val_accuracy: 0.9939 - val_loss: 0.0160
Epoch 96/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 1.0000 - loss: 0.0072 - val_accuracy: 1.0000 - val_loss: 8.0163e-04
Epoch 97/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 348ms/step - accuracy: 0.9934 - loss: 0.0176 - val_accuracy: 0.9905 - val_loss: 0.0225
Epoch 98/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 2.4438e-06
Epoch 99/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 61s 359ms/step - accuracy: 0.9973 - loss: 0.0097 - val_accuracy: 0.9912 - val_loss: 0.0269
Epoch 100/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 244us/step - accuracy: 1.0000 - loss: 0.0060 - val_accuracy: 0.9167 - val_loss: 0.0611


In [9]:
# Load the best saved model
model.load_weights('best_vgg16_model.keras')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

/home/esidserver/miniforge3/envs/tf/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 62 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - accuracy: 0.8789 - loss: 0.3517
Validation Accuracy: 0.8740
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 188ms/step - accuracy: 0.8869 - loss: 0.3335
Test Accuracy: 0.8824
